# Analysis on the Mannville Group Dataset

In [ ]:
import glob
import lasio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.preprocessing import binarize
from colorsys import hsv_to_rgb
from random import randint, uniform
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
%matplotlib inline

## Download the data

well LAS files from <https://dataunderground.org/dataset/athabasca> 

These LAS files are pulled to get the datum elevations, well names and convert all tops to TVDSS. You must download the data from the link above and set the path in `get_well_data` to the path of the LAS files

In [ ]:

def get_well_data(path):
    """
    This function gets the Mannville data from the LAS files
    Including, UWI, datum, ground elevation, reference elevation, and elevation units
    :param path: path to the unzipped well LAS files
    :return: dataframe with well data
    """
    wells = glob.glob(PATH)
    uwi =[]
    datum = []
    ground = []
    eref = []
    units = []
    failure = 0
    for well in wells[0:]:
        curve = lasio.read(well)
        try:
            da = curve.params['DATM'].value
            gl = curve.params['GL'].value
            u = curve.well['UWI'].value
            units.append(curve.params['GL'].unit)
            eref.append(curve.params['EREF'].value)
            uwi.append(u.replace('W400', 'W4/0'))
            datum.append(da)
            ground.append(gl)
        except:
            failure +=1
            print(well)
    print(str(failure) + ' logs failed to parse')
    well_dict = pd.read_csv(r'mann_well_dict.csv')
    dataframe = pd.DataFrame({"UWI":uwi, "DATUM":datum, "GL":ground, 'EREF':eref, 'UNIT':units})
    dataframe['datum'] = np.where(dataframe.UNIT=='F', dataframe.EREF*0.304, dataframe.EREF)
    # drop wells with no surface datum
    bad_datums = ['00/07-11-082-07W4/0',
        '00/10-08-095-21W4/0',
        '00/11-29-094-21W4/0',
        'AA/05-01-096-11W4/0',
        'AA/06-09-095-10W4/0',
        'AA/06-31-096-10W4/0',
        'AA/08-25-096-13W4/0',
        'AA/10-33-097-06W4/0',
        'AA/15-36-096-11W4/0',
        'AB/07-12-093-10W4/0',
        'AB/10-18-096-10W4/0']
    bad_wells = dataframe[dataframe['UWI'].isin(bad_datums)].index.values
    dataframe.drop(bad_wells, inplace=True)
    dataframe.to_csv('mann_dirty_LAS.csv')
    return dataframe

def clean_well_data():
    """
    This function cleans the output from get_well_data and saves it to a csv
    :param none:
    :return: none
    """
    well_dict = pd.read_csv(r'mann_well_dict.csv')
    dataframe = pd.read_csv('mann_dirty_LAS.csv')
    new_df = pd.merge(dataframe, well_dict,  how='left', left_on=['UWI'], right_on = ['UWI'])
    new_df.dropna(inplace=True)
    tops = pd.read_csv(r"mannvillegrp_picks.csv") #read in the top data
    tops.rename(columns={'Pick':'MD'}, inplace=True)
    new_df = pd.merge(tops, new_df,  how='left', left_on=['SitID'], right_on = ['SitID'])
    not_nullDF = new_df.loc[new_df['UWI'].notnull()]
    not_nullDF['TVDSS'] = (not_nullDF.datum-not_nullDF.MD).values
    not_nullDF.to_csv('mannville_cleaned.csv')

################################################    

def sample_splitter(dataframe, fraction, randomseed):
    """
    Splits a dataframe into train and test subsets
    :param dataframe: Tops dataframe
    :param fraction: The fraction of tops to use for validation
    :param randomseed: The random seed for random sampling
    :return: training and testing dataframes
    """
    test = dataframe.sample(frac=fraction, random_state=randomseed)
    test_idx = test.index.values
    train = dataframe.drop(test_idx)
    return train, test

# ALS factorization modified from
# https://github.com/mickeykedia/Matrix-Factorization-ALS/blob/master/ALS%20Python%20Implementation.py
# here items are the formation and users are the well
def runALS(A, R, n_factors, n_iterations, lambda_):
    """
    Runs Alternating Least Squares algorithm in order to calculate matrix.
    :param A: User-Item Matrix with ratings
    :param R: User-Item Matrix with 1 if there is a rating or 0 if not
    :param n_factors: How many factors each of user and item matrix will consider
    :param n_iterations: How many times to run algorithm
    :param lambda_: Regularization parameter
    :return: users, items from ALS
    """
    n, m = A.shape
    np.random.seed(86)
    Users = 5 * np.random.rand(n, n_factors,)
    Items = 5 * np.random.rand(n_factors, m)

    def get_error(A, Users, Items, R):
        # This calculates the MSE of nonzero elements
        return np.sum((R * (A - np.dot(Users, Items))) ** 2) / np.sum(R)

    MAE_List = []

    print("Starting Iterations")
    for iteration in range(n_iterations):
        for i, Ri in enumerate(R):
            Users[i] = np.linalg.solve(
                np.dot(Items, np.dot(np.diag(Ri), Items.T))
                + lambda_ * np.eye(n_factors),
                np.dot(Items, np.dot(np.diag(Ri), A[i].T)),
            ).T
        for j, Rj in enumerate(R.T):
            Items[:, j] = np.linalg.solve(
                np.dot(Users.T, np.dot(np.diag(Rj), Users))
                + lambda_ * np.eye(n_factors),
                np.dot(Users.T, np.dot(np.diag(Rj), A[:, j])),
            )
        MAE_List.append(get_error(A, Users, Items, R))
    return Users, Items



In [ ]:
def cross_validation_error(dataframe, random_seed, latent_vectors, n_iters, reg):
    """
    Splits a dataframe into 4 different folds for cross validation MAE and RMSE
    :param dataframe: Tops dataframe
    :param random_seed: The random seed for random sampling
    :param latent_vectors: Number of latent vectors for matrix factorization
    :param n_iters: Number of iterations to run ALS
    :param reg: Lamda regularization value
    :return: MAE and RMSE error values for each fold
    """
    np.random.seed(random_seed)
    block_1  = dataframe.sample(n=dataframe.shape[0]//4, random_state=random_seed).index.values
    tops2 = dataframe.drop(block_1)
    block_2 = tops2.sample(n=dataframe.shape[0]//4, random_state=random_seed).index.values
    tops3 = tops2.drop(block_2)
    block_3 = tops3.sample(n=dataframe.shape[0]//4, random_state=random_seed).index.values
    tops4 = tops3.drop(block_3)
    block_4 = tops4.sample(n=dataframe.shape[0]//4, random_state=random_seed).index.values
    blocks = [block_1, block_2, block_3, block_4]
    CV_MAE = []
    CV_MSE = []
    for block in blocks:
        validate = dataframe.loc[block]
        main_group = dataframe.drop(block)
        print(f'Validating on {block.shape[0]} tops')
        D_df = main_group.pivot_table("TVDSS", "Formation", "SitID").fillna(0)#pivot table to move into sparse matrix land
        R = D_df.values
        A = binarize(R) 

        
        U, Vt = runALS(R, A, latent_vectors, n_iters, reg)

        recommendations = np.dot(U, Vt) #get the recommendations

        recsys = pd.DataFrame(
            data=recommendations[0:, 0:], index=D_df.index, columns=D_df.columns
        ) #results

        newDF = recsys.T
        newDF.reset_index(inplace=True)

        flat_preds = pd.DataFrame(recsys.unstack()).reset_index()

        new_df = pd.merge(validate, flat_preds,  how='left', left_on=['SitID','Formation'], right_on = ['SitID','Formation'])

        new_df.rename(columns={0:'SS_pred'}, inplace=True)

        cleanDF = new_df.dropna()

        cleanDF['signed_error'] = (cleanDF['TVDSS'] - cleanDF['SS_pred'])

        CV_MAE.append(MAE(cleanDF.TVDSS.values, cleanDF.SS_pred.values))
        CV_MSE.append(np.sqrt(MSE(cleanDF.TVDSS.values, cleanDF.SS_pred.values)))

    return CV_MAE, CV_MSE

def cross_validation(dataframe, random_seed, latent_vectors, n_iters, reg):
    """
    Splits a dataframe into 4 different folds for cross validation
    :param dataframe: Tops dataframe
    :param random_seed: The random seed for random sampling
    :param latent_vectors: Number of latent vectors for matrix factorization
    :param n_iters: Number of iterations to run ALS
    :param reg: Lamda regularization value
    :return: dataframe with predictions and MAE error
    """
    full = []
    block_1 = dataframe.sample(
        n=dataframe.shape[0] // 4, random_state=random_seed
    ).index.values
    tops2 = dataframe.drop(block_1)
    block_2 = tops2.sample(
        n=dataframe.shape[0] // 4, random_state=random_seed
    ).index.values
    tops3 = tops2.drop(block_2)
    block_3 = tops3.sample(
        n=dataframe.shape[0] // 4, random_state=random_seed
    ).index.values
    tops4 = tops3.drop(block_3)
    block_4 = tops4.sample(
        n=dataframe.shape[0] // 4, random_state=random_seed
    ).index.values
    blocks = [block_1, block_2, block_3, block_4]
    CV_MAE = []
    bk = 1
    for block in blocks:
        print(f'Starting Block {bk}')
        validate = dataframe.loc[block]
        main_group = dataframe.drop(block)
        print(f'Validating on {block.shape[0]} tops')
        D_df = main_group.pivot_table("TVDSS", "Formation", "SitID").fillna(0)#pivot table to move into sparse matrix land
        R = D_df.values
        A = binarize(R) 

        
        U, Vt = runALS(R, A, latent_vectors, n_iters, reg)

        recommendations = np.dot(U, Vt) #get the recommendations

        recsys = pd.DataFrame(
            data=recommendations[0:, 0:], index=D_df.index, columns=D_df.columns
        ) #results

        newDF = recsys.T
        newDF.reset_index(inplace=True)

        flat_preds = pd.DataFrame(recsys.unstack()).reset_index()

        newDF = recsys.T
        newDF.reset_index(inplace=True)

        flat_preds = pd.DataFrame(recsys.unstack()).reset_index()

        new_df = pd.merge(validate, flat_preds,  how='left', left_on=['SitID','Formation'], right_on = ['SitID','Formation'])

        new_df.rename(columns={0:'SS_pred'}, inplace=True)

        cleanDF = new_df.dropna()

        cleanDF['signed_error'] = (cleanDF['TVDSS'] - cleanDF['SS_pred'])
        cleanDF['Block'] = [bk]*cleanDF.shape[0]
        well_locs = pd.read_csv(r'well_lat_lng.csv')
        full.append(cleanDF.merge(well_locs[['lat', 'lng', 'SitID']], on='SitID'))
        CV_MAE.append(MAE(cleanDF.TVDSS.values, cleanDF.SS_pred.values))
        bk += 1
    output = pd.concat(full)
    return output

def cross_validation_wells(dataframe, random_seed, latent_vectors, n_iters, reg):
    """
    Splits a dataframe into 4 different folds for cross validation on each well
    :param dataframe: Tops dataframe
    :param random_seed: The random seed for random sampling
    :param latent_vectors: Number of latent vectors for matrix factorization
    :param n_iters: Number of iterations to run ALS
    :param reg: Lamda regularization value
    :return: dataframe with error for each wells predictions
    """
    cv_wells = []
    block_1 = dataframe.sample(
        n=dataframe.shape[0] // 4, random_state=random_seed
    ).index.values
    tops2 = dataframe.drop(block_1)
    block_2 = tops2.sample(
        n=dataframe.shape[0] // 4, random_state=random_seed
    ).index.values
    tops3 = tops2.drop(block_2)
    block_3 = tops3.sample(
        n=dataframe.shape[0] // 4, random_state=random_seed
    ).index.values
    tops4 = tops3.drop(block_3)
    block_4 = tops4.sample(
        n=dataframe.shape[0] // 4, random_state=random_seed
    ).index.values
    blocks = [block_1, block_2, block_3, block_4]
    f = 0
    for block in blocks:
        validate = dataframe.loc[block]
        main_group = dataframe.drop(block)
        print(f'Validating on {block.shape[0]} tops')
        D_df = main_group.pivot_table("TVDSS", "Formation", "SitID").fillna(0)#pivot table to move into sparse matrix land
        R = D_df.values
        A = binarize(R) 

        
        U, Vt = runALS(R, A, latent_vectors, n_iters, reg)

        recommendations = np.dot(U, Vt) #get the recommendations

        recsys = pd.DataFrame(
            data=recommendations[0:, 0:], index=D_df.index, columns=D_df.columns
        ) #results

        newDF = recsys.T
        newDF.reset_index(inplace=True)

        flat_preds = pd.DataFrame(recsys.unstack()).reset_index()

        new_df = pd.merge(validate, flat_preds,  how='left', left_on=['SitID','Formation'], right_on = ['SitID','Formation'])

        new_df.rename(columns={0:'SS_pred'}, inplace=True)

        cleanDF = new_df.dropna()

        cleanDF['signed_error'] = (cleanDF['TVDSS'] - cleanDF['SS_pred'])
        well_locs = pd.read_csv(r'well_lat_lng.csv')
        locationDF = cleanDF.merge(well_locs[['lat', 'lng', 'SitID']], on='SitID')
        aypi = []
        well_mae = []
        well_rmse = []
        east = []
        north = []
        fold = []
        
        print(f'foldno is {f}')
        for well in locationDF.SitID.unique():
            aypi.append(well)
            well_mae.append(locationDF[locationDF.SitID == well].signed_error.abs().mean())
            well_rmse.append(np.sqrt(MSE(locationDF[locationDF.SitID == well].TVDSS,
                                         locationDF[locationDF.SitID == well].SS_pred)))
            east.append(locationDF[locationDF.SitID == well].lng.values[0])
            north.append(locationDF[locationDF.SitID == well].lat.values[0])
            fold.append(f)
        by_wellDF = pd.DataFrame({'SitID':aypi, 'Well_MAE':well_mae, 'well_rmse':well_rmse, 'Longitude':east, 'Latitude':north,
                                 'foldno':fold})
        cv_wells.append(by_wellDF)
        f+=1
    return cv_wells

In [ ]:
PATH = '*.las' # path to the las files

In [ ]:
dirty = get_well_data(PATH)

In [ ]:
clean_well_data()

# Below are the predictions, above is cleaning

In [ ]:
tops = pd.read_csv(r"mannville_cleaned.csv", index_col=[0]) #read in the top data
print(tops.shape)
tops.dropna(inplace=True)
print(tops.shape)
tops = tops[tops.Quality >=0]
print(tops.shape)

In [ ]:
training, testing = sample_splitter(tops, 0.1, 86)

print(f'Training size is {len(training)} tops, and test size is {len(testing)} tops')

QC_D_df = training.pivot_table("TVDSS", "Formation", "SitID").fillna(0)#pivot table to move into sparse matrix land
QC_R = QC_D_df.values
QC_A = binarize(QC_R) 

In [ ]:
print(f'{round(((QC_D_df == 0).astype(int).sum().sum())/((QC_D_df == 0).astype(int).sum().sum()+(QC_D_df != 0).astype(int).sum().sum()),3)*100} percent of the tops are missing')

In [ ]:
print(f'There are {len(tops.UWI.unique())} wells and {len(tops.Formation.unique())} tops')

# Grid Search

In [ ]:
%%capture
grid_search = {}
els = []
nsits = []
regulars = []
for L in range(1, 11):
    for n_it in range(10, 450, 10):
        for reg in [0.001, 0.01, 0.1, 1, 10]:
            grid_search[L, n_it, reg] = np.mean(
                cross_validation_error(tops, 86, L, n_it, reg)
            )
            els.append(L)
            nsits.append(n_it)
            regulars.append(reg)
            print(L, n_it, reg)

In [ ]:
#L, its, lbda = min(grid_search, key=grid_search.get)
L, its, lbda = [3, 100, 0.1]

In [ ]:
mae, rmse = cross_validation_error(tops, 86, L, its, lbda)
print(mae, rmse)

In [ ]:
print(np.mean(mae), np.mean(rmse))

In [ ]:
cv_DF = cross_validation(tops, 86, L, its, lbda)

# Adding stratigraphy
Here we add in the ordered stratigraphy so we can make sense of things

In [ ]:
strat_order = [1000,2000,3000,4000,5000,6000,7000,8000,9000,9500,10000,11000,12000,13000,14000]

And we assign some colors. The original output from this cell is saved as `mann_color_palette.csv`

In [ ]:
color_list = []
for i in range(len(strat_order)):
    if i ==0: #mannville
        h = uniform(23/360, 33/360) # Select random green'ish hue from hue wheel
        s = uniform(0.8, 1)
        v = uniform(0.2, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])
    elif 0<i<=4: #t61 to t31
        h = uniform(83/360, 158/360) 
        s = uniform(0.8, 1)
        v = uniform(0.3, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])

    elif 4<i<=5: #clw_wab
        h = uniform(23/360, 33/360) 
        s = uniform(0.8, 1)
        v = uniform(0.3, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])

    elif 5<i<= 6: #t21
        h = uniform(83/360, 158/360) 
        s = uniform(0.8, 1)
        v = uniform(0.3, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])

    elif 6<i<= 7: #e20
        h = uniform(23/360, 33/360) 
        s = uniform(0.8, 1)
        v = uniform(0.3, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])

    elif 7<i<= 8: #t15
        h = uniform(83/360, 158/360) 
        s = uniform(0.8, 1)
        v = uniform(0.3, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])
        
    elif 8<i<= 9: #e14
        h = uniform(23/360, 33/360) 
        s = uniform(0.8, 1)
        v = uniform(0.3, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])
        
    elif 9<i<= 10: #t11
        h = uniform(83/360, 158/360) 
        s = uniform(0.8, 1)
        v = uniform(0.3, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])
    
    elif 10<i<= 11: #t10.5
        h = uniform(83/360, 158/360) 
        s = uniform(0.8, 1)
        v = uniform(0.3, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])
        
    elif 11<i<= 12: #e10
        h = uniform(23/360, 33/360) 
        s = uniform(0.8, 1)
        v = uniform(0.2, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])
        
    elif 12<i<= 13: #mcmurray
        h = uniform(50/360, 60/360) 
        s = uniform(0.8, 1)
        v = uniform(0.2, 1)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])
        
    else: #PZ
        h = uniform(300/360, 320/360) 
        s = uniform(0.2, 1)
        v = uniform(0.3, 0.5)
        r, g, b = hsv_to_rgb(h, s, v)
        color_list.append([r,g,b])

#colors_to_plot = dict(zip(strat_order, color_list))
#pd.DataFrame(colors_to_plot).to_csv('mann_color_palette.csv', index=False)
colors_to_plot =  pd.read_csv('mann_color_palette.csv').to_dict(orient='list')

In [ ]:
for block in cv_DF['Block'].unique():
    locationDF = cv_DF[cv_DF['Block'] == block]
    mae_errors = []
    n_holdout = []
    formed = []
    colored = []
    stdev = []
    n_train = []
    rms_errors = []
    for formation in strat_order:
        form = locationDF[locationDF['Formation'] == formation]
        if form.TVDSS.values.shape[0] > 0:
            mae_errors.append(round(MAE(form.TVDSS.values, form.SS_pred.values),1))
            formed.append(formation)
            stdev.append(np.std(form.signed_error))
            n_holdout.append(form.shape[0])
            n_train.append(tops[tops.Formation == formation].shape[0]-form.shape[0])

            rms_errors.append(np.sqrt(MSE(form.TVDSS.values, form.SS_pred.values)))
        else:
            mae_errors.append(np.nan)
            formed.append(formation)
            stdev.append(np.std(form.signed_error))
            n_train.append(tops[tops.Formation == formation].shape[0]-form.shape[0])
            n_holdout.append(form.shape[0])
            rms_errors.append(np.nan)
    table_1 = pd.DataFrame({'Formation':formed, 'n_train': n_train, 'n_holdout':n_holdout, 'MAE':mae_errors,
                 'RMSE':rms_errors, 'Std':stdev})
    nums = dict(zip(formed, n_holdout))
    errers = dict(zip(formed, mae_errors))
    val = dict(zip(formed, n_holdout))
    table_1.to_csv('mann_Appendix_1_block_'+str(block)+'.csv')

In [ ]:
vc = tops.Formation.value_counts(sort=True)
fm_mapping = {1000:'Mannville Group', 2000:'t61', 3000:'t51', 4000:'t41', 5000:'t31', 6000:'Clearwater/Wabiskaw', 7000:'t21',
              8000:'e20', 9000:'t15', 9500:'e14', 10000:'t11',11000:'t10.5', 12000:'e10', 13000:'McMurray Formation', 
              14000:'Paleozoic'}

In [ ]:
fig1 = plt.figure(figsize=(10,10))
for i in enumerate(strat_order):
    width = 1
    height = 1
    lims = (0, 10)
    
    ax1 = fig1.add_subplot(111, aspect='equal')
    ax1.add_patch(
        patches.Rectangle((0, i[0]*-1), width, height, color=colors_to_plot[str(i[1])]))
    ax1.annotate(fm_mapping[i[1]], (1.5, -1*i[0]+0.25), fontsize=12)
    ax1.annotate(vc[vc.index ==i[1]].values,  (6, -1*i[0]+0.25), fontsize=12)
    plt.ylim(-59,1)
    plt.xlim(lims)
plt.tight_layout()
plt.axis('off')
#plt.savefig('mann_strat_column.pdf')

In [ ]:
# this is output from the folds, compressed by hand in excel 
errors = pd.read_csv('mann_errors.csv') 
plt.rcParams.update({'font.size': 6})
grid = plt.GridSpec(2, 1, wspace=0.4, hspace=0.1)
fig = plt.figure(figsize=(2.5, 5))

upper = plt.subplot(grid[0, :])

for formation in strat_order:
    subset = errors[errors.Formation == formation]
    upper.scatter(subset['n_holdout']+subset['n_train'], subset['MAE'], 
                color=colors_to_plot[str(formation)], alpha=0.75, s=15)
    upper.scatter(subset['n_holdout.1']+subset['n_train.1'], subset['MAE.1'],
                color=colors_to_plot[str(formation)], alpha=0.75, s=15)
    upper.scatter(subset['n_holdout.2']+subset['n_train.2'], subset['MAE.2'],
                color=colors_to_plot[str(formation)], alpha=0.75, s=15)
    up = upper.scatter(subset['n_holdout.3']+subset['n_train.3'], subset['MAE.3'],
                color=colors_to_plot[str(formation)], alpha=0.75, s=15)
upper.semilogx()
upper.semilogy()
upper.set_ylabel('MAE (m)', fontsize=6)
upper.set_title('Error and Number of Picks', fontsize=6)

#upper.set_xticklabels([10**1,10**2, 10**3],fontsize=6,)
#upper.set_yticklabels([10^-1,10**0, 10**1, 10**2, 10**3],fontsize=6)#plt.yticks(fontsize=6)
upper.set_xlim(100,5000)
upper.set_ylim(1,110)


lower = plt.subplot(grid[1, :])

for formation in strat_order:
    subset = errors[errors.Formation == formation]
    lower.scatter(subset['n_holdout']+subset['n_train'], subset['RMSE'], 
                color=colors_to_plot[str(formation)], alpha=0.75, s=15)
    lower.scatter(subset['n_holdout.1']+subset['n_train.1'], subset['RMSE.1'],
                color=colors_to_plot[str(formation)], alpha=0.75, s=15)
    lower.scatter(subset['n_holdout.2']+subset['n_train.2'], subset['RMSE.2'],
                color=colors_to_plot[str(formation)], alpha=0.75, s=15)
    lo = lower.scatter(subset['n_holdout.3']+subset['n_train.3'], subset['RMSE.3'],
                color=colors_to_plot[str(formation)], alpha=0.75, s=15)
lower.semilogx()
lower.semilogy()
lower.set_xlabel('Number of picks per top', fontsize=6)
lower.set_ylabel('RMSE (m)', fontsize=6)
#lower.set_title('Root Mean Squared Error per Formation', fontsize=6)
#lower.set_xticklabels([10**1,10**2, 10**3],fontsize=6,)
#lower.set_yticklabels([10^-1,10**0, 10**1, 10**2, 10**3],fontsize=6)#plt.yticks(fontsize=6)
lower.set_xlim(100,5000)
lower.set_ylim(1,110)


plt.savefig('mann_MAE_formation.pdf')

# Error by well

In [ ]:
masterDF = pd.concat(cross_validation_wells(tops, 86, L, its, 0.1))
masterDF.to_csv('mann_error_map.csv')

# Validation with SOTA Green's Function spline interpolation

In [ ]:
import verde as vd
import itertools
from sklearn.model_selection import KFold

In [ ]:
tops = pd.read_csv(r"mannville_cleaned.csv", index_col=[0]) #read in the top data
tops.dropna(inplace=True)
tops = tops[tops.Quality >=0]
tops.rename(columns={"TVDSS": "SS"}, inplace=True)
well_locations = pd.read_csv(r'well_lat_lng.csv')

In [ ]:
def verde_error(location_dataframe, tops_df, grid_spacing, random_seed):
    """
    Spatially interpolates between sample locations and measures error
    using four-fold cross validation
    :param location_dataframe: dataframe with locations
    :param tops_df: dataframe with tops data
    :param grid_spacing: grid square size in degrees
    :param random_seed: random seed for reproducibility
    :return: dictionary with MAE and RMSE for the 4 folds along with average
    """
    data_dict={"Top":[], "MAE 1":[], "MAE 2":[], "MAE 3":[], "MAE 4":[], "MAE avg":[], "RMSE 1":[], "RMSE 2":[],
              "RMSE 3":[], "RMSE 4":[], "RMSE avg":[]}
    region = vd.get_region((location_dataframe.lng.values, location_dataframe.lat.values))
    spacing = grid_spacing
    cross_validator = KFold(n_splits=4, random_state=random_seed)
    for formation in strat_order:
        subset_DF = tops_df[tops_df.Formation == formation]
        subset_DF = subset_DF.merge(location_dataframe[["lng", "lat", "SitID"]], on="SitID")
        if subset_DF.shape[0] < 4:
            data_dict['Top'].append(formation)
            data_dict['MAE 1'].append(np.nan)
            data_dict['MAE 2'].append(np.nan)
            data_dict['MAE 3'].append(np.nan)
            data_dict['MAE 4'].append(np.nan)
            data_dict['RMSE 1'].append(np.nan)
            data_dict['RMSE 2'].append(np.nan)
            data_dict['RMSE 3'].append(np.nan)
            data_dict['RMSE 4'].append(np.nan)
            data_dict['MAE avg'].append(np.nan)
            data_dict['RMSE avg'].append(np.nan)
            
        else:
            coordinates = (subset_DF.lng.values, subset_DF.lat.values)
            spline = vd.Spline()
            RMSE = vd.cross_val_score(spline, coordinates, subset_DF.SS.values, 
                       cv=cross_validator, scoring="neg_root_mean_squared_error")*-1
            MAE = vd.cross_val_score(spline, coordinates, subset_DF.SS.values, 
                       cv=cross_validator, scoring="neg_mean_absolute_error")*-1
            data_dict['Top'].append(formation)
            data_dict['MAE 1'].append(MAE[0])
            data_dict['MAE 2'].append(MAE[1])
            data_dict['MAE 3'].append(MAE[2])
            data_dict['MAE 4'].append(MAE[3])
            data_dict['RMSE 1'].append(RMSE[0])
            data_dict['RMSE 2'].append(RMSE[1])
            data_dict['RMSE 3'].append(RMSE[2])
            data_dict['RMSE 4'].append(RMSE[3])
            data_dict['MAE avg'].append(np.mean(MAE))
            data_dict['RMSE avg'].append(np.mean(RMSE))
    return data_dict


In [ ]:
pd.DataFrame(verde_error(well_locations, tops, 0.01, 86))

In [ ]:
def verde_spatial_error(location_dataframe, tops_df, block_size, random_seed):
    """
    Spatially interpolates between sample locations and measures error
    using blocked spatial four-fold cross validation
    :param location_dataframe: dataframe with locations
    :param tops_df: dataframe with tops data
    :param block_size: size of the spatial block 
    :param random_seed: random seed for reproducibility
    :return: dictionary with MAE and RMSE for the 4 folds along with average
    """
    data_dict={"Top":[], "MAE 1":[], "MAE 2":[], "MAE 3":[], "MAE 4":[], "MAE avg":[], "RMSE 1":[], "RMSE 2":[],
              "RMSE 3":[], "RMSE 4":[], "RMSE avg":[]}
    region = vd.get_region((location_dataframe.lng.values, location_dataframe.lat.values))
    cross_validator = vd.BlockKFold(spacing=block_size, n_splits=4, random_state=random_seed, shuffle=True,
                                   balance=True)
    for formation in strat_order:
        subset_DF = tops_df[tops_df.Formation == formation]
        subset_DF = subset_DF.merge(location_dataframe[["lat", "lng", "SitID"]], on="SitID")
        if subset_DF.shape[0] < 4:
            data_dict['Top'].append(formation)
            data_dict['MAE 1'].append(np.nan)
            data_dict['MAE 2'].append(np.nan)
            data_dict['MAE 3'].append(np.nan)
            data_dict['MAE 4'].append(np.nan)
            data_dict['RMSE 1'].append(np.nan)
            data_dict['RMSE 2'].append(np.nan)
            data_dict['RMSE 3'].append(np.nan)
            data_dict['RMSE 4'].append(np.nan)
            data_dict['MAE avg'].append(np.nan)
            data_dict['RMSE avg'].append(np.nan)
            
        else:
            coordinates = (subset_DF.lng.values, subset_DF.lat.values)
            spline = vd.Spline()
            RMSE = vd.cross_val_score(spline, coordinates, subset_DF.SS.values, 
                       cv=cross_validator, scoring="neg_root_mean_squared_error")*-1
            CMAE = vd.cross_val_score(spline, coordinates, subset_DF.SS.values, 
                       cv=cross_validator, scoring="neg_mean_absolute_error")*-1
            data_dict['Top'].append(formation)
            data_dict['MAE 1'].append(CMAE[0])
            data_dict['MAE 2'].append(CMAE[1])
            data_dict['MAE 3'].append(CMAE[2])
            data_dict['MAE 4'].append(CMAE[3])
            data_dict['RMSE 1'].append(RMSE[0])
            data_dict['RMSE 2'].append(RMSE[1])
            data_dict['RMSE 3'].append(RMSE[2])
            data_dict['RMSE 4'].append(RMSE[3])
            data_dict['MAE avg'].append(np.mean(CMAE))
            data_dict['RMSE avg'].append(np.mean(RMSE))
    return data_dict

In [ ]:
pd.DataFrame(verde_spatial_error(well_locations, tops, 0.1, 86))

In [ ]:
def recsys_spatial_error(location_dataframe, tops_df, block_size, random_seed, latent, iterations, reg):
    """
    Runs ALS recsys and measures error
    using blocked spatial four-fold cross validation
    :param location_dataframe: dataframe with locations
    :param tops_df: dataframe with tops data
    :param block_size: size of the spatial block 
    :param random_seed: random seed for reproducibility
    :param latent: number of latent factors for RecSys
    :param iterations: number of iterations for RecSys
    :param reg: L2 regularization value
    :return: dictionary with MAE and RMSE for the 4 folds along with average
    """
    ss_df = tops_df.merge(location_dataframe[["lat", "lng", "SitID"]], on="SitID")
    coordinates = (ss_df.lng.values, ss_df.lat.values)
    kfold = vd.BlockKFold(spacing=block_size, n_splits=4, random_state=random_seed, shuffle=True, balance=True)
    feature_matrix = np.transpose(coordinates)
    data_dict={"Top":[], "N_train 1":[], "N_test 1":[],"N_train 2":[], "N_test 2":[],"N_train 3":[], 
           "N_test 3":[],"N_train 4":[], "N_test 4":[], "MAE 1":[], "MAE 2":[], "MAE 3":[], 
           "MAE 4":[], "MAE avg":[], "RMSE 1":[], "RMSE 2":[],
              "RMSE 3":[], "RMSE 4":[], "RMSE avg":[]}
    for formation in strat_order:
        full = []
        CV_MAE = []
        CV_MSE = []
        n_train = []
        n_test = []
        for train, test in kfold.split(feature_matrix):
            test_set = ss_df.loc[test]
            validate = test_set[test_set.Formation == formation]
            validate_index = validate.index.values
            main_group = ss_df.drop(validate_index)

            D_df = main_group.pivot_table("SS", "Formation", "API").fillna(
                0
            )  # pivot table to move into sparse matrix land
            R = D_df.values
            A = binarize(R)

            U, Vt = runALS(R, A, latent, iterations, reg)

            recommendations = np.dot(U, Vt)  # get the recommendations

            recsys = pd.DataFrame(
                data=recommendations[0:, 0:], index=D_df.index, columns=D_df.columns
            )  # results

            newDF = recsys.T
            newDF.reset_index(inplace=True)

            flat_preds = pd.DataFrame(recsys.unstack()).reset_index()

            new_df = pd.merge(
                validate,
                flat_preds,
                how="left",
                left_on=["API", "Formation"],
                right_on=["API", "Formation"],
            )
            new_df.rename(columns={0: "SS_pred"}, inplace=True)
            cleanDF = new_df.dropna()
            cleanDF["signed_error"] = cleanDF["SS"] - cleanDF["SS_pred"]

            full.append(cleanDF.merge(location_dataframe[["lat", "lng", "API"]], on="API"))
            CV_MAE.append(MAE(cleanDF.SS.values - ssmin, cleanDF.SS_pred.values - ssmin))
            CV_MSE.append(
                np.sqrt(MSE(cleanDF.SS.values - ssmin, cleanDF.SS_pred.values - ssmin))
            )
            n_train.append(len(main_group[main_group.Formation == "F2WC"]))
            n_test.append(len(validate))
        data_dict['Top'].append(formation)
        data_dict['N_train 1'].append(n_train[0])
        data_dict['N_train 2'].append(n_train[1])
        data_dict['N_train 3'].append(n_train[2])
        data_dict['N_train 4'].append(n_train[3])
        data_dict['N_test 1'].append(n_test[0])
        data_dict['N_test 2'].append(n_test[1])
        data_dict['N_test 3'].append(n_test[2])
        data_dict['N_test 4'].append(n_test[3])

        data_dict['MAE 1'].append(CV_MAE[0])
        data_dict['MAE 2'].append(CV_MAE[1])
        data_dict['MAE 3'].append(CV_MAE[2])
        data_dict['MAE 4'].append(CV_MAE[3])
        data_dict['RMSE 1'].append(CV_MSE[0])
        data_dict['RMSE 2'].append(CV_MSE[1])
        data_dict['RMSE 3'].append(CV_MSE[2])
        data_dict['RMSE 4'].append(CV_MSE[3])
        data_dict['MAE avg'].append(np.mean(CV_MAE))
        data_dict['RMSE avg'].append(np.mean(CV_MSE))
    return data_dict

In [ ]:
ss_df = tops.merge(well_locations[["lat", "lng", "SitID"]], on="SitID")
coordinates = (ss_df.lng.values, ss_df.lat.values)
kfold = vd.BlockKFold(spacing=0.1, n_splits=4, random_state=86, shuffle=True, balance=True)
feature_matrix = np.transpose(coordinates)
for train, test in kfold.split(feature_matrix):
    print(ss_df.loc[test].shape)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(ss_df.loc[train].lng, ss_df.loc[train].lat, c='k', alpha=0.2)
plt.scatter(ss_df.loc[test].lng, ss_df.loc[test].lat, c='r', alpha=0.5)

In [ ]:
def recsys_spatial_error(location_dataframe, tops_df, block_size, random_seed, latent, iterations, reg):
    """
    Runs ALS recsys and measures error
    using blocked spatial four-fold cross validation
    :param location_dataframe: dataframe with locations
    :param tops_df: dataframe with tops data
    :param block_size: size of the spatial block 
    :param random_seed: random seed for reproducibility
    :param latent: number of latent factors for RecSys
    :param iterations: number of iterations for RecSys
    :param reg: L2 regularization value
    :return: dictionary with MAE and RMSE for the 4 folds along with average
    """
    ss_df = tops_df.merge(location_dataframe[["lat", "lng", "SitID"]], on="SitID")
    coordinates = (ss_df.lng.values, ss_df.lat.values)
    kfold = vd.BlockKFold(spacing=block_size, n_splits=4, random_state=random_seed, shuffle=True, balance=True)
    feature_matrix = np.transpose(coordinates)
    data_dict={"Top":[], "N_train 1":[], "N_test 1":[],"N_train 2":[], "N_test 2":[],"N_train 3":[], 
           "N_test 3":[],"N_train 4":[], "N_test 4":[], "MAE 1":[], "MAE 2":[], "MAE 3":[], 
           "MAE 4":[], "MAE avg":[], "RMSE 1":[], "RMSE 2":[],
              "RMSE 3":[], "RMSE 4":[], "RMSE avg":[]}
    for formation in strat_order:
        if len(ss_df[ss_df.Formation == formation]) < 1:
            print(f"Not enough picks to proceed... Skipping {formation}")
        else:
                  
            print(f"Starting with {formation}")
            full = []
            CV_MAE = []
            CV_MSE = []
            n_train = []
            n_test = []
            for train, test in kfold.split(feature_matrix):
                test_set = ss_df.loc[test]
                validate = test_set[test_set.Formation == formation]
                validate_index = validate.index.values
                main_group = ss_df.drop(validate_index)

                D_df = main_group.pivot_table("SS", "Formation", "SitID").fillna(
                    0
                )  # pivot table to move into sparse matrix land
                R = D_df.values
                A = binarize(R)

                U, Vt = runALS(R, A, latent, iterations, reg)

                recommendations = np.dot(U, Vt)  # get the recommendations

                recsys = pd.DataFrame(
                    data=recommendations[0:, 0:], index=D_df.index, columns=D_df.columns
                )  # results

                newDF = recsys.T
                newDF.reset_index(inplace=True)

                flat_preds = pd.DataFrame(recsys.unstack()).reset_index()

                new_df = pd.merge(
                    validate,
                    flat_preds,
                    how="left",
                    left_on=["SitID", "Formation"],
                    right_on=["SitID", "Formation"],
                )
                new_df.rename(columns={0: "SS_pred"}, inplace=True)
                cleanDF = new_df.dropna()
                cleanDF["signed_error"] = cleanDF["SS"] - cleanDF["SS_pred"]

                full.append(cleanDF.merge(location_dataframe[["lat", "lng", "SitID"]], on="SitID"))
                CV_MAE.append(MAE(cleanDF.SS.values, cleanDF.SS_pred.values))
                CV_MSE.append(
                    np.sqrt(MSE(cleanDF.SS.values, cleanDF.SS_pred.values))
                )
                n_train.append(len(main_group[main_group.Formation == formation]))
                n_test.append(len(validate))
        data_dict['Top'].append(formation)
        data_dict['N_train 1'].append(n_train[0])
        data_dict['N_train 2'].append(n_train[1])
        data_dict['N_train 3'].append(n_train[2])
        data_dict['N_train 4'].append(n_train[3])
        data_dict['N_test 1'].append(n_test[0])
        data_dict['N_test 2'].append(n_test[1])
        data_dict['N_test 3'].append(n_test[2])
        data_dict['N_test 4'].append(n_test[3])

        data_dict['MAE 1'].append(CV_MAE[0])
        data_dict['MAE 2'].append(CV_MAE[1])
        data_dict['MAE 3'].append(CV_MAE[2])
        data_dict['MAE 4'].append(CV_MAE[3])
        data_dict['RMSE 1'].append(CV_MSE[0])
        data_dict['RMSE 2'].append(CV_MSE[1])
        data_dict['RMSE 3'].append(CV_MSE[2])
        data_dict['RMSE 4'].append(CV_MSE[3])
        data_dict['MAE avg'].append(np.mean(CV_MAE))
        data_dict['RMSE avg'].append(np.mean(CV_MSE))
    return data_dict

In [ ]:
pd.DataFrame(recsys_spatial_error(well_locations, tops, 0.1, 86, 3, 100, 0.1))